IMPORTS

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
from prog import scheduler as sch

In [ ]:
# save current working directory as homey
# homey = os.path.abspath(os.path.dirname(__file__))
homey = os.getcwd() # works in jupyter notebook

In [ ]:
# set directory paths
dataPath = os.path.join(homey, 'data')
progPath = os.path.join(homey, 'prog')
simPath = os.path.join(homey, 'FB_Sim')
forcPath = os.path.join(simPath, 'ForecastRedoux')
# this should probably be moved out of the simulator and into the main sql area, where that ends up
sqlPath = os.path.join(forcPath, 'SQL')

In [ ]:
# set paths to excel files
forecastFilename = os.path.join(dataPath, 'RegularForecast.xlsx')
mfgCentersFilename = os.path.join(dataPath, 'MfgCenters.xlsx')
moFilename = os.path.join(dataPath, 'MOs.xlsx')
laborAvailFilename = os.path.join(dataPath, 'LaborAvailablePerDay.xlsx')
leadFilename = os.path.join(dataPath, 'LeadTimes.xlsx')

QUERIES

In [ ]:
sys.path.insert(0, forcPath)

In [ ]:
import ForecastMain as fm
import ForecastAPI as fa

In [ ]:
# pull the usual FB_Sim queries
# fa.run_queries(queryPath=sqlPath, dataPath=dataPath)

GET DATA

In [ ]:
# save mfgCenters as df, includes MFG Center assignments and Setup/labor time estimates
mfgCenters = pd.read_excel(mfgCentersFilename, header=0)

In [ ]:
# save current Manufacture Orders
modf = pd.read_excel(moFilename, header=0)

In [ ]:
# save lead time estimates
leadTimes = pd.read_excel(leadFilename, header=0)

In [ ]:
# this is a placeholder for a calculation of start to finish time for a build.
# just using it for earliest schedule date right now.
orderRunTime = 7

MAKE DATE LIST

In [ ]:
# make a date list with labor availability
dateList = sch.create_date_list(dailyLabor=11)

CREATE IDEAL SCHEDULE

In [ ]:
# prep mo list with mfg centers and labor estimates
preppedMOdf = sch.pre_schedule_prep(modf=modf.copy(), mfgCenters=mfgCenters.copy())

In [ ]:
# run the auto schedule to get an ideal schedule by priority
moLinesLabor = sch.run_auto_schedule(moLinesLabor=preppedMOdf.copy(), dateList=dateList.copy())

In [ ]:
# use the last scheduled FG in an order to save an ideal schedule
idealSchedule = moLinesLabor.drop_duplicates('ORDER', keep='last')

RUN THE SIM

In [ ]:
# replace the schedule dates on the MO order lines with the new dates for those orders
newMOdf = pd.merge(modf.copy(), idealSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')
newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()
newMOdf.drop(labels='NewDate', axis=1, inplace=True)

In [ ]:
# run the new MO schedule through the FB_Sim to find phantom orders
orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())

GET SCHEDULE LIMITS

In [ ]:
orderLeads = sch.get_earliest_leads(orderTimeline=orderTimeline.copy(),
                                    leadTimes=leadTimes.copy(),
                                    dateList=dateList.copy(),
                                    orderRunTime=orderRunTime)

ANALYZE SCHEDULE

In [ ]:
newMOdf = sch.analyze_schedule(newMOdf=newMOdf.copy(),
                               orderLeads=orderLeads.copy(),
                               modf=modf.copy(),
                               mfgCenters=mfgCenters.copy(),
                               dateList=dateList.copy(),
                               orderRunTime=orderRunTime,
                               leadTimes=leadTimes.copy())

In [ ]:
tempMOdf = newMOdf.sort_values(by=['ORDER','DATESCHEDULED'], ascending=[True, True]).copy()

In [ ]:
tempMOdf.drop_duplicates('ORDER', keep='first', inplace=True)

In [ ]:
checkSched = pd.merge(tempMOdf[['ORDER','DATESCHEDULED']].copy(), orderLeads[['ORDER','EarliestScheduleDate']].copy(), how='left', on='ORDER')

In [ ]:
checkSched['TimeDiff'] = np.nan

In [ ]:
for each in range(0, len(checkSched)):
    if checkSched['DATESCHEDULED'].iat[each] < checkSched['EarliestScheduleDate'].iat[each]:
        checkSched['TimeDiff'] = 'here'
        print(checkSched['ORDER'].iat[each])

In [ ]:
len(checkSched.dropna())

CREATE NEW SCHEDULE

In [ ]:
# save a new copy of the modf with longest leads added
leadMOdf = pd.merge(modf.copy(), orderLeads[['ORDER','EarliestScheduleDate']].copy(), how='left', on='ORDER')

In [ ]:
moLinesLabor = sch.pre_schedule_prep(modf=leadMOdf, mfgCenters=mfgCenters.copy())

In [ ]:
outputSchedule = sch.sched_with_date_limits(orderPriority=moLinesLabor.copy(),
                                            dateList=dateList.copy())

In [ ]:
# use the last scheduled FG in an order to save a new schedule
newSchedule = outputSchedule.drop_duplicates('ORDER', keep='last').copy()

RUN THE SIM

In [ ]:
# replace the schedule dates on the MO order lines with the new dates for those orders
newMOdf = pd.merge(modf.copy(), newSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')

In [ ]:
newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()

In [ ]:
newMOdf.drop(labels='NewDate', axis=1, inplace=True)

In [ ]:
# run the new MO schedule through the FB_Sim to find phantom orders
orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())

GET SCHEDULE LIMITS

In [ ]:
# get a fresh list of earliest leads per order from the recent sim run
freshLeads = sch.get_earliest_leads(orderTimeline=orderTimeline.copy(),
                                    leadTimes=leadTimes.copy(),
                                    dateList=dateList.copy(),
                                    orderRunTime=orderRunTime)
# combine it with any previous lists to get the last schedule date per order
orderLeads = sch.combine_order_leads(oldLeads=orderLeads.copy(), newLeads=freshLeads.copy())